In [1]:
import pandas as pd 
import numpy as np
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,LSTM,Embedding,Bidirectional

In [2]:
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [3]:
df=pd.read_csv('C:/Users/ASUS ROG/OneDrive/Documents/bbc-text.csv')

In [4]:
df.head(5)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [5]:
a=df['category'].value_counts()
print(a)

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64


In [6]:
len(df)

2225

In [7]:
train_size = int(len(df)*.7)
test_size = int(len(df)*.2)+1

In [8]:
print(train_size)
print(test_size)

1557
446


In [9]:
def split(df, train_size):
    train = df[:train_size]
    test = df[train_size:]
    return train, test

In [10]:
train_cat, test_cat = split(df['category'], train_size)
train_text, test_text = split(df['text'], train_size)

In [11]:
max_features = 1000
tokenize = Tokenizer(num_words=max_features,char_level=False)

In [12]:
tokenize.fit_on_texts(train_text) 
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [13]:
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [14]:
num_classes = np.max(y_train) + 1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [15]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (1557, 1000)
x_test shape: (668, 1000)
y_train shape: (1557, 5)
y_test shape: (668, 5)


In [38]:
model = Sequential()
model.add(Dense(512,activation='relu', input_shape=(max_features,)))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu', input_shape=(max_features,)))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu', input_shape=(max_features,)))
model.add(Dropout(0.5))
model.add(Dense((num_classes),activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [41]:
history = model.fit(x_train, y_train,
                    batch_size=16,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/5
88/88 [==============================] - 1s 16ms/step - loss: 0.0570 - accuracy: 0.9829 - val_loss: 0.2156 - val_accuracy: 0.9551
Epoch 2/5
88/88 [==============================] - 1s 13ms/step - loss: 0.0233 - accuracy: 0.9936 - val_loss: 0.2328 - val_accuracy: 0.9423
Epoch 3/5
88/88 [==============================] - 1s 13ms/step - loss: 0.0203 - accuracy: 0.9936 - val_loss: 0.2964 - val_accuracy: 0.9295
Epoch 4/5
88/88 [==============================] - 1s 13ms/step - loss: 0.0455 - accuracy: 0.9850 - val_loss: 0.2879 - val_accuracy: 0.9295
Epoch 5/5
88/88 [==============================] - 1s 13ms/step - loss: 0.0228 - accuracy: 0.9921 - val_loss: 0.2758 - val_accuracy: 0.9423


# text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_text.iloc[i][:50], "...")
    print('Actual label:' + test_cat.iloc[i])
    print("Predicted label: " + predicted_label + "\n")